In [22]:
import pandas as pd
import numpy as np
df=pd.read_csv("combined.csv")

def convertToFloat(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0
    else:
        
        return float(x)
df["rain"]=df["Rain"].apply(convertToFloat)
# import datetime
df.pop("Rain")
aldf=df[df.District=="Alangayam"]
# df["month"]=df["Date"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
# df["year"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)
# df["day"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").day)

In [26]:
dataset=aldf[["rain"]].values
dataset

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [27]:
np.random.seed(7)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset=scaler.fit_transform(dataset)

In [38]:
def CreateDataset(dataset,lookback):
    DataX=[]
    DataY=[]
    for i in range(len(dataset)-lookback-1):
        DataX.append(dataset[i:(lookback+i),0])
        DataY.append(dataset[(lookback+i),0])
    return np.array(DataX),np.array(DataY)

In [70]:
trainsize=3287
testsize=365
train, test = dataset[0:trainsize,:], dataset[trainsize:len(dataset),:]

In [88]:
look_back=60
trainX,trainY=CreateDataset(train,look_back)
testX,testY=CreateDataset(test,look_back)


(3256, 30)

In [89]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [90]:
trainX

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.01322751, ..., 0.        ,
         0.        , 0.        ]]])

In [91]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=200, batch_size=1, verbose=2)

Epoch 1/200
3226/3226 - 3s - loss: 0.0038
Epoch 2/200
3226/3226 - 3s - loss: 0.0036
Epoch 3/200
3226/3226 - 3s - loss: 0.0036
Epoch 4/200
3226/3226 - 3s - loss: 0.0036
Epoch 5/200
3226/3226 - 3s - loss: 0.0035
Epoch 6/200
3226/3226 - 3s - loss: 0.0035
Epoch 7/200
3226/3226 - 2s - loss: 0.0035
Epoch 8/200
3226/3226 - 2s - loss: 0.0035
Epoch 9/200
3226/3226 - 3s - loss: 0.0035
Epoch 10/200
3226/3226 - 3s - loss: 0.0035
Epoch 11/200
3226/3226 - 3s - loss: 0.0035
Epoch 12/200
3226/3226 - 3s - loss: 0.0034
Epoch 13/200
3226/3226 - 3s - loss: 0.0034
Epoch 14/200
3226/3226 - 3s - loss: 0.0034
Epoch 15/200
3226/3226 - 3s - loss: 0.0034
Epoch 16/200
3226/3226 - 3s - loss: 0.0034
Epoch 17/200
3226/3226 - 3s - loss: 0.0033
Epoch 18/200
3226/3226 - 3s - loss: 0.0033
Epoch 19/200
3226/3226 - 3s - loss: 0.0033
Epoch 20/200
3226/3226 - 3s - loss: 0.0033
Epoch 21/200
3226/3226 - 3s - loss: 0.0032
Epoch 22/200
3226/3226 - 2s - loss: 0.0032
Epoch 23/200
3226/3226 - 2s - loss: 0.0032
Epoch 24/200
3226/32

In [92]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [93]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [95]:
from sklearn.metrics import mean_squared_error


ypred=testPredict.reshape(304)
ty=testY.reshape(304)

In [96]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(ty,ypred)

7.394569770703216